# Assignment 2: Language Modelling in Hangman

Student Name:

Student ID:

## General info

<b>Due date</b>: Wednesday, 9 April 2025 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python on Google Colab" page on Canvas (under Modules>Resources) for information on how to use Google Colab to use Jupyter Notebook. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages, <b> except </b> Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. You must export your Colab notebook and submit it to the LMS. If your notebook doesn't run, you will lose marks. <b> You should use the Python version that's on Google Colab (3.11)</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. You <b> must not </b> use artifial intelligence tools for doing the assignment, as all work submitted by a student should be their own, as clarified <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [24]:
from typing import Callable

In [9]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [10]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [21]:
interactive = True #todo change this to false

<b>For your testing:</b>

You can play the game interactively using the following command:

In [12]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [13]:
import nltk
from nltk.corpus import brown
import numpy as np

nltk.download('brown')
np.random.seed(1)

[nltk_data] Downloading package brown to
[nltk_data]     /home/higherorder2/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [14]:

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
# Get all words from brown corpus and filter them
words = brown.words()
filtered_words = [word.lower() for word in words if word.isalpha()]
unique_words = list(set(filtered_words))

# Shuffle the words randomly
np.random.shuffle(unique_words)

# Split into test and training sets
test_set = unique_words[:1000]
training_set = unique_words[1000:]
###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [15]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(training_set) == 39234) # added test
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [22]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

Starting hangman game. Target is _ _ _ _ _ length 5
You have 8 attempts remaining.

Enter your guess:
Guess is a
Sorry, try again.
You have 7 attempts remaining.

Enter your guess:
Guess is e
Sorry, try again.
You have 6 attempts remaining.

Enter your guess:
Guess is e
Already guessed this before.
You have 5 attempts remaining.

Enter your guess:
Guess is i
Good guess: _ i _ _ _
You have 5 attempts remaining.

Enter your guess:
Guess is t
Good guess: _ i _ t _
You have 5 attempts remaining.

Enter your guess:
Guess is c
Sorry, try again.
You have 4 attempts remaining.

Enter your guess:
Guess is f
Sorry, try again.
You have 3 attempts remaining.

Enter your guess:
Guess is l
Good guess: l i _ t _
You have 3 attempts remaining.

Enter your guess:
Guess is s
Good guess: l i s t s
Congratulations, you won.


### Question 2 (0.5 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
def test_guesser(guesser: Callable, test: list[str]) -> float:
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
        
        Args:
            guesser (Callable): A function that takes a mask string, set of guessed chars and returns a char guess
            test (list[str]): List of words to test the guesser on
            
        Returns:
            float: Average number of incorrect guesses across all test words
            
        Example:
            >>> def simple_guesser(mask: str, guessed: set[str], **kwargs) -> str:
            ...     return 'e'  # Always guess 'e'
            >>> test_words = ['hello', 'world']
            >>> test_guesser(simple_guesser, test_words)
            2.5  # Average number of wrong guesses
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [29]:
def random_guesser(mask: str, guessed: set[str], **kwargs) -> str:
    """
    Just return a random character from the English alphabet
    
    Args:
        mask (str): The current state of the word with unguessed letters as '_'
        guessed (set[str]): Set of characters that have already been guessed
        **kwargs: Additional keyword arguments (not used)
        
    Returns:
        str: A random unguessed character from a-z
        
    Example:
        >>> mask = "h_ll_"
        >>> guessed = {'h', 'l'} 
        >>> random_guesser(mask, guessed)
        'e'  # Returns random letter that isn't 'h' or 'l'
    """
    
    ###
    # Your answer BEGINS HERE
    ###
    # Get all letters from a-z
    alphabet = set('abcdefghijklmnopqrstuvwxyz')
    
    # need to remove already guessed letters
    available = alphabet - guessed
    
    # Return random letter from remaining available letters
    return np.random.choice(list(available))
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser, test_set)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = expire
Number of mistakes made by the random guesser = 18

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.771


<b>For your testing:</b>

In [30]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method (use the second answer space). Note that it takes `unigram_counts` as an additional argument.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
unigram_counts = None

###
# Your answer BEGINS HERE

###
# Initialize empty dictionary for character counts
unigram_counts = {}

# Loop through each word in training set
for word in training_set:
    # Loop through each character in word
    for char in word:
        # Increment count for this character
        if char in unigram_counts:
            unigram_counts[char] += 1
        else:
            unigram_counts[char] = 1


###
# Your answer ENDS HERE
###

def unigram_guesser(mask: str, guessed: set, unigram_counts=unigram_counts) -> str:
    """Guess next letter based on unigram probabilities.
    
    Uses unigram language model to select the most probable character that hasn't been guessed yet.
    
    P(c) = count(c) / N, where N is total character count
    
    Args:
        mask: Current game state showing revealed letters and underscores for hidden letters
        guessed: Set of letters that have already been guessed
        unigram_counts: Dictionary mapping characters to their counts in training data
    
    Returns:
        str: Most probable unguessed character
        
    Example:
        >>> mask = "h_ll_"
        >>> guessed = {'h','l'} 
        >>> unigram_counts = {'e':10, 'o':8, 'a':5}
        >>> unigram_guesser(mask, guessed, unigram_counts)
        'e'
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    # Find character with highest count that hasn't been guessed yet
    max_count = 0
    best_char = None
    
    for char, count in unigram_counts.items():
        # Skip if already guessed
        if char in guessed:
            continue
            
        if count > max_count:
            max_count = count
            best_char = char
            
    return best_char

    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_guesser, test_set)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)


Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.32


10.32

<b>For your testing:</b>

In [34]:
#h Assert result is 10.32
assert(result == 10.32)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [35]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###

unigram_counts_by_length = {}

for word in training_set:
    length = len(word)
    
    # Initialize dictionary for this length if it doesn't exist
    if length not in unigram_counts_by_length:
        unigram_counts_by_length[length] = {}
    
    # Count each character in the word
    for char in word:
        if char in unigram_counts_by_length[length]:
            unigram_counts_by_length[length][char] += 1
        else:
            unigram_counts_by_length[length][char] = 1

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask: str, guessed: set[str], unigram_counts_by_length: dict[int, dict[str, int]]=unigram_counts_by_length, unigram_counts: dict[str, int]=unigram_counts) -> str:
    """
    Makes a guess for the next character based on unigram frequencies conditioned on word length.
    Falls back to unconditioned unigram frequencies for unseen word lengths.

    Example:
        >>> mask = "h_ll_"  # 5 letter word
        >>> guessed = {"h", "l"}
        >>> unigram_length_guesser(mask, guessed)  # Returns most likely unguessed char for 5-letter words
        'e'

    Args:
        mask: The current state of the word with _ for unknown letters
        guessed: Set of letters that have already been guessed
        unigram_counts_by_length: Dict mapping word lengths to character frequency dicts
        unigram_counts: Dict mapping characters to their frequencies (fallback)

    Returns:
        The most likely unguessed character
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    length = len(mask)
    
    # If we haven't seen this word length before, fall back to regular unigram counts
    if length not in unigram_counts_by_length:
        max_count = 0
        best_char = None
        
        for char, count in unigram_counts.items():
            if char not in guessed and count > max_count:
                max_count = count
                best_char = char
                
        return best_char
    
    # Use length-specific unigram counts
    max_count = 0
    best_char = None
    
    for char, count in unigram_counts_by_length[length].items():
        if char not in guessed and count > max_count:
            max_count = count
            best_char = char
            
    return best_char

    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser, test_set)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.256


<b>For your testing:</b>

In [37]:
assert(result > 5 and result < 15)
assert(result == 10.256) #h added test

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Note**:
- When backing-off to the unigram language model, you **must use the vanilla unigram language model that you have built in Q3**. Do not use the length-based unigram language model, or any fancy variant, or you will lose marks.
- You should build a **standard bigram language model**; i.e. do not do anything complicated like a bidirectional bigram language model.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [68]:
def bigram_sum(prev):
      values = [val for key, val in forward_bigram_dict[prev].items()]
      total = sum(values)
      return total

assert bigram_sum('$') == len(training_set)

In [93]:
bigram_counts = None

###
# Your answer BEGINS HERE
###
"""#h
first go through each word in the corpus
    for each word, for the charactor before it, code the frequency of the charactor after it, 
    unless start, which goes to a sperate thing

example
    leet, chad
      ^
    forward_bigram_dict {
    '$': {l:1},
    'l': {e:1},
    'e' :{e:1,t:1},
      }

questions
  do we need a special end symbol?
training_set

"""
from collections import defaultdict
forward_bigram_dict = defaultdict(lambda: defaultdict(int))

for word in training_set:
    for i in range(0,len(word)):
        c = word[i]
        if i ==0:
            forward_bigram_dict['$'][c] +=1
        else:
            prev_c = word[i-1]
            assert i - 1 >-1
            forward_bigram_dict[prev_c][c] +=1


###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask: str, guessed: set[str], bigram_counts=bigram_counts) -> str:
    """Guess the next letter using a bigram language model.
    
    The probability of a character c following character p is:
    P(c|p) = count(p,c) / count(prev) # should be summed prev from the bigram dictionary
    
    hell_

    
    Where:
    - count(p,c) is the number of times character c follows p in training data
    - count(p) is total number of characters that follow p
    
    Examples:
        >>> mask = "h_ll_"
        >>> guessed = {'h', 'l'} 
        >>> bigram_guesser(mask, guessed)
        'e'  # High probability of 'e' following 'h' in "hello"
        
        >>> mask = "_at" 
        >>> guessed = {'t', 'a'}
        >>> bigram_guesser(mask, guessed)
        'c'  # 'c' likely to start "cat" based on training data

        C_t_
        cat

        hello
        h_ll_
        he
        le


    """
    ###
    # Your answer BEGINS HERE
    ###
    """
    #h probably could optimise but hey
    process
    
        

    example:
    $h_ll_
      ^

    have to divide by total 

    """
    def bigram_sum(prev):
      values = [val for key, val in forward_bigram_dict[prev].items()]
      total = sum(values)
      return total
    
    unigram_total = sum([val for key, val in unigram_counts.items()])

    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    char_prob_dict = defaultdict(int)
    # print(mask)
    mask = ['$']+ mask
    for i in range(1,len(mask)):
      c = mask[i]
      #technically its count
      if c == '_':
        """
        example:
        $h_ll_
         ^
        """
        for guess in alphabet:
          if guess in guessed:
            continue
          prev = mask[i-1]
          if prev == '_':
            probability = unigram_counts[guess] / unigram_total
            assert 0<probability<1
          else:
            p_prev = bigram_sum(prev)
            p_c = forward_bigram_dict[prev][guess]
            # if p_c == 0:
            #    print('true')
            #smoothing?
            probability = 0
            if not p_c == 0:
              probability =  p_c /p_prev

          char_prob_dict[guess] += probability
        

    largest_prob = 0
    largest_guess = None
    for guess, prob in char_prob_dict.items():
      if prob > largest_prob and guess not in guessed:
        largest_prob = prob
        largest_guess = guess
    return largest_guess
    
    ###
    # Your answer ENDS HERE
    ###



result = test_guesser(bigram_guesser, test_set)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.679


### Question 6 (2.0 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 2.0 mark if x < 6.0
* 1.5 mark if 6.0 <= x < 6.5
* 1.0 mark if 6.5 <= x < 7.0
* 0.5 mark if 7.0 <= x < 8.0
* 0.0 mark if x >= 8.0

Note:
* When testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its _average_ performance, as there will be some variance to its performance depending on the training/test split.
* Your code must run under 2 minutes on Colab (free account) with GPU; program that runs longer than that will be terminated and you will score 0.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [ ]:
###
# Your answer BEGINS HERE
###

def my_amazing_ai_guesser(mask, guessed):
     return random_guesser(mask, guessed)

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser, test_set)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE



##### Your answer ENDS HERE